In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sonata_data = pd.read_csv('sonata_data.csv')
soul_data = pd.read_csv('soul_data.csv')
spark_data = pd.read_csv('spark_data.csv')

In [4]:
print("Sonata shape: ", sonata_data.shape)
print("Soul shape: ", soul_data.shape)
print("Spark shape: ", spark_data.shape)

Sonata shape:  (535041, 12)
Soul shape:  (797843, 12)
Spark shape:  (402956, 12)


In [5]:
spark_data.head()

,timestamp,canId,dlc,data0,data1,data2,data3,data4,data5,data6,data7,flag
0,1.513920e+09,04C1,8,00,CC,80,5E,52,08,00,00,Benign
1,1.513920e+09,04C7,3,10,00,00,-1,-1,-1,-1,-1,Benign
2,1.513920e+09,01E1,7,00,00,00,00,00,00,00,-1,Benign
3,1.513920e+09,00C1,8,00,F9,05,41,02,85,8B,91,Benign
4,1.513920e+09,00C5,8,03,52,0F,1D,C3,F4,03,D4,Benign


In [6]:
flag_values = pd.DataFrame(columns=['Sonata', 'Soul', 'Spark'])
flag_values['Sonata'] = sonata_data['flag'].value_counts()
flag_values['Soul'] = soul_data['flag'].value_counts()
flag_values['Spark'] = spark_data['flag'].value_counts()
flag_values

,Sonata,Soul,Spark
flag,,,
Benign,468527,717489,366510
Flooding,32422,33141,22587
Fuzzy,18118,39812,5812
Malfunction,15974,7401,8047


In [7]:
sonata_data = sonata_data.drop(columns=['timestamp'])
soul_data = soul_data.drop(columns=['timestamp'])
spark_data = spark_data.drop(columns=['timestamp'])
spark_data.head()

,canId,dlc,data0,data1,data2,data3,data4,data5,data6,data7,flag
0,04C1,8,00,CC,80,5E,52,08,00,00,Benign
1,04C7,3,10,00,00,-1,-1,-1,-1,-1,Benign
2,01E1,7,00,00,00,00,00,00,00,-1,Benign
3,00C1,8,00,F9,05,41,02,85,8B,91,Benign
4,00C5,8,03,52,0F,1D,C3,F4,03,D4,Benign


In [8]:
X_sonata = sonata_data.drop(columns=['flag'])
Y_sonata = sonata_data['flag']
Y_sonata = pd.DataFrame(Y_sonata, columns=['flag'])

X_soul = soul_data.drop(columns=['flag'])
Y_soul = soul_data['flag']
Y_soul = pd.DataFrame(Y_soul, columns=['flag'])

X_spark = spark_data.drop(columns=['flag'])
Y_spark = spark_data['flag']
Y_spark = pd.DataFrame(Y_spark, columns=['flag'])

In [9]:
X_spark.head()

,canId,dlc,data0,data1,data2,data3,data4,data5,data6,data7
0,04C1,8,00,CC,80,5E,52,08,00,00
1,04C7,3,10,00,00,-1,-1,-1,-1,-1
2,01E1,7,00,00,00,00,00,00,00,-1
3,00C1,8,00,F9,05,41,02,85,8B,91
4,00C5,8,03,52,0F,1D,C3,F4,03,D4


In [10]:
Y_sonata.head()

,flag
0,Benign
1,Benign
2,Benign
3,Benign
4,Benign


In [11]:
label_encoder = LabelEncoder()
combined_flags = pd.concat([sonata_data['flag'], soul_data['flag'], spark_data['flag']], axis=0)
label_encoder.fit(combined_flags)

Y_sonata['label_multiclass'] = label_encoder.transform(Y_sonata['flag'])
Y_sonata['label_binary'] = Y_sonata['flag'].apply(lambda x: 0 if x == 'Benign' else 1)
Y_sonata.drop(columns=['flag'], inplace=True)

Y_soul['label_multiclass'] = label_encoder.transform(Y_soul['flag'])
Y_soul['label_binary'] = Y_soul['flag'].apply(lambda x: 0 if x == 'Benign' else 1)
Y_soul.drop(columns=['flag'], inplace=True)

Y_spark['label_multiclass'] = label_encoder.transform(Y_spark['flag'])
Y_spark['label_binary'] = Y_spark['flag'].apply(lambda x: 0 if x == 'Benign' else 1)
Y_spark.drop(columns=['flag'], inplace=True)

In [12]:
label_mapping = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))
print("Label Mapping (Encoded -> Class):")
for encoded_label, class_name in label_mapping.items():
    print(f"{encoded_label} -> {class_name}")

Label Mapping (Encoded -> Class):
0 -> Benign
1 -> Flooding
2 -> Fuzzy
3 -> Malfunction


In [13]:
Y_sonata.head()

,label_multiclass,label_binary
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [14]:
print(X_sonata.dtypes)

canId    object
dlc       int64
data0    object
data1    object
data2    object
data3    object
data4    object
data5    object
data6    object
data7    object
dtype: object


In [15]:
def preprocess_hex_values(data):
    for col in data.columns:
        if col.startswith('data') or col == 'canId':
            data[col] = data[col].apply(lambda x: int(x, 16))
    return data

X_sonata = preprocess_hex_values(X_sonata)
X_soul = preprocess_hex_values(X_soul)
X_spark = preprocess_hex_values(X_spark)

In [16]:
X_spark.head()

,canId,dlc,data0,data1,data2,data3,data4,data5,data6,data7
0,1217,8,0,204,128,94,82,8,0,0
1,1223,3,16,0,0,-1,-1,-1,-1,-1
2,481,7,0,0,0,0,0,0,0,-1
3,193,8,0,249,5,65,2,133,139,145
4,197,8,3,82,15,29,195,244,3,212


In [17]:
X_train_sonata, X_test_sonata, Y_train_sonata, Y_test_sonata = train_test_split(X_sonata, Y_sonata, test_size=0.2, random_state=42, stratify=Y_sonata)
X_train_soul, X_test_soul, Y_train_soul, Y_test_soul = train_test_split(X_soul, Y_soul, test_size=0.2, random_state=42, stratify=Y_soul)
X_train_spark, X_test_spark, Y_train_spark, Y_test_spark = train_test_split(X_spark, Y_spark, test_size=0.2, random_state=42, stratify=Y_spark)


In [18]:
class CanBusDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X.values if isinstance(X, pd.DataFrame) else np.array(X)
        self.Y = Y.values if isinstance(Y, pd.DataFrame) else np.array(Y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_values = self.X[idx].astype(np.float32)
        Y_values = self.Y[idx].astype(np.float32)
        return torch.tensor(X_values).to(device), torch.tensor(Y_values[0]).to(device), torch.tensor(Y_values[1]).to(device)


In [19]:
train_dataset_sonata = CanBusDataset(X_train_sonata, Y_train_sonata)
test_dataset_sonata = CanBusDataset(X_test_sonata, Y_test_sonata)

train_loader_sonata = DataLoader(train_dataset_sonata, batch_size=512, shuffle=True)
test_loader_sonata = DataLoader(test_dataset_sonata, batch_size=512, shuffle=False)

train_dataset_soul = CanBusDataset(X_train_soul, Y_train_soul)
test_dataset_soul = CanBusDataset(X_test_soul, Y_test_soul)

train_loader_soul = DataLoader(train_dataset_soul, batch_size=512, shuffle=True)
test_loader_soul = DataLoader(test_dataset_soul, batch_size=512, shuffle=False)

train_dataset_spark = CanBusDataset(X_train_spark, Y_train_spark)
test_dataset_spark = CanBusDataset(X_test_spark, Y_test_spark)

train_loader_spark = DataLoader(train_dataset_spark, batch_size=512, shuffle=True)
test_loader_spark = DataLoader(test_dataset_spark, batch_size=512, shuffle=False)

In [20]:
for idx, data in enumerate(train_loader_sonata):
    datas = data[0]
    labels = data[1]
    print("Datas shape:", datas.shape)
    print("Labels shape:", labels.shape)
    break

Datas shape: torch.Size([512, 10])
Labels shape: torch.Size([512])


In [21]:
len(train_dataset_sonata), len(test_dataset_sonata)

(428032, 107009)

In [31]:
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, network_type):
        super(LSTMNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.network_type = network_type

    def forward(self, X):
        if len(X.shape) == 2:
            X = X.unsqueeze(1)
        batch_size = X.size(0)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        carry = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        output, (hidden, carry) = self.lstm(X, (hidden, carry))
        output = self.fc(output[:, -1])
        if self.network_type == 'binary':
          return torch.sigmoid(output)
        else:
          return output

    def fit(self, train_dataloader, test_dataloader, epochs, optimizer, loss_function):
        train_losses = []
        test_losses = []

        for epoch in range(epochs):
            self.train()
            train_loss = 0.0
            for X_batch, y_batch_multiclass, y_batch_binary in train_dataloader:
                optimizer.zero_grad()
                y_pred = self.forward(X_batch).squeeze()

                if self.network_type == 'binary':
                    y_batch = y_batch_binary.float().squeeze()
                else:
                    y_batch = y_batch_multiclass.long().squeeze()

                loss = loss_function(y_pred, y_batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            train_losses.append(train_loss / len(train_dataloader))

            self.eval()
            test_loss = 0.0
            with torch.no_grad():
                for X_batch, y_batch_multiclass, y_batch_binary in test_dataloader:
                    y_pred = self.forward(X_batch).squeeze()

                    if self.network_type == 'binary':
                      y_batch = y_batch_binary.float().squeeze()
                    else:
                      y_batch = y_batch_multiclass.long().squeeze()

                    loss = loss_function(y_pred, y_batch)
                    test_loss += loss.item()
            test_losses.append(test_loss / len(test_dataloader))

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")

        return train_losses, test_losses

    def predict(self, dataloader):
        all_outputs = []
        self.eval()
        with torch.no_grad():
            for X_batch, _, _ in dataloader:
                X_batch = X_batch.to(device)
                outputs = self.forward(X_batch)
                if self.network_type == 'binary':
                  predicted = (outputs > 0.5).float()
                else:
                  _, predicted = torch.max(outputs.data, 1)
                all_outputs.extend(predicted.cpu().numpy())
        return np.array(all_outputs)

In [23]:
def verify_parameters_binary_classification(y_test, y_pred_tab, parameters):
    results = pd.DataFrame(columns=['Accuracy', 'Recall', 'F1 Score', 'FPR', 'FNR'])
    for i in range(len(parameters)):
        accuracy = accuracy_score(y_test['label_binary'], y_pred_tab[i])
        recall = recall_score(y_test['label_binary'], y_pred_tab[i], average='macro')
        f1 = f1_score(y_test['label_binary'], y_pred_tab[i], average='macro')

        tn, fp, fn, tp = confusion_matrix(y_test['label_binary'], y_pred_tab[i]).ravel()
        fpr = fp / (fp + tn)
        fnr = fn / (fn + tp)

        results.loc[str(parameters[i])] = [accuracy, recall, f1, fpr, fnr]

    return results

In [76]:
def calculate_scores_binary_classification(y_test, y_pred_tab):
    results = pd.DataFrame(columns=['Accuracy', 'Recall', 'F1 Score', 'FPR', 'FNR'])

    classes = list(label_mapping.items())
    classes = classes[1:]

    for encoded_label, class_name in classes:
        y_test_modified = y_test[(y_test['label_multiclass'] == encoded_label) | (y_test['label_multiclass'] == 0)]
        y_pred_modified = y_pred_tab[(y_test['label_multiclass'] == encoded_label) | (y_test['label_multiclass'] == 0)]

        accuracy = accuracy_score(y_test_modified['label_binary'], y_pred_modified)
        recall = recall_score(y_test_modified['label_binary'], y_pred_modified)
        f1 = f1_score(y_test_modified['label_binary'], y_pred_modified)

        cm = confusion_matrix(y_test_modified['label_binary'], y_pred_modified, labels=[0, 1])

        tn, fp, fn, tp = cm.ravel()
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

        results.loc[class_name] = [accuracy, recall, f1, fpr, fnr]

    results.loc['Average'] = results.mean()

    accuracy = accuracy_score(y_test['label_binary'], y_pred_tab)
    recall = recall_score(y_test['label_binary'], y_pred_tab)
    f1 = f1_score(y_test['label_binary'], y_pred_tab)
    cm = confusion_matrix(y_test['label_binary'], y_pred_tab, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    results.loc['Overall'] = [accuracy, recall, f1, fpr, fnr]


    return results

In [77]:
def calculate_scores_multiclass_classification(y_test, y_pred_tab):
    results = pd.DataFrame(columns=['Accuracy', 'Recall', 'F1 Score', 'FPR', 'FNR'])

    cm = confusion_matrix(y_test['label_multiclass'], y_pred_tab, labels=[0, 1, 2, 3])

    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (tp + fp + fn)

    accuracy = np.sum(tp) / np.sum(cm)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)

    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)

    for encoded_label, class_name in label_mapping.items():
        results.loc[class_name] = [accuracy, recall[encoded_label], f1[encoded_label], fpr[encoded_label], fnr[encoded_label]]


    results.loc['Average'] = results.mean()

    return results

In [26]:
input_size = X_train_sonata.shape[1]
input_size

10

In [60]:
lstm = LSTMNetwork(input_size=10, hidden_size=512, output_size=1, num_layers=1, network_type='binary').to(device)
loss = lstm.fit(train_loader_sonata, test_loader_sonata, 20, optim.Adam(lstm.parameters(), lr=0.0001), nn.BCELoss())
predictions = lstm.predict(test_loader_sonata)

Epoch 1/20 - Train Loss: 0.1269, Test Loss: 0.0346
Epoch 2/20 - Train Loss: 0.0174, Test Loss: 0.0119
Epoch 3/20 - Train Loss: 0.0082, Test Loss: 0.0069
Epoch 4/20 - Train Loss: 0.0053, Test Loss: 0.0047
Epoch 5/20 - Train Loss: 0.0038, Test Loss: 0.0035
Epoch 6/20 - Train Loss: 0.0029, Test Loss: 0.0029
Epoch 7/20 - Train Loss: 0.0023, Test Loss: 0.0023
Epoch 8/20 - Train Loss: 0.0019, Test Loss: 0.0020
Epoch 9/20 - Train Loss: 0.0015, Test Loss: 0.0017
Epoch 10/20 - Train Loss: 0.0012, Test Loss: 0.0014
Epoch 11/20 - Train Loss: 0.0010, Test Loss: 0.0011
Epoch 12/20 - Train Loss: 0.0008, Test Loss: 0.0010
Epoch 13/20 - Train Loss: 0.0006, Test Loss: 0.0009
Epoch 14/20 - Train Loss: 0.0005, Test Loss: 0.0008
Epoch 15/20 - Train Loss: 0.0004, Test Loss: 0.0015
Epoch 16/20 - Train Loss: 0.0004, Test Loss: 0.0014
Epoch 17/20 - Train Loss: 0.0003, Test Loss: 0.0015
Epoch 18/20 - Train Loss: 0.0003, Test Loss: 0.0014
Epoch 19/20 - Train Loss: 0.0002, Test Loss: 0.0013
Epoch 20/20 - Train L

In [78]:
df_scores = calculate_scores_binary_classification(Y_test_sonata, predictions)

In [79]:
df_scores

,Accuracy,Recall,F1 Score,FPR,FNR
Flooding,0.999990,1.000000,0.999923,0.000011,0.000000
Fuzzy,0.999928,0.998344,0.999034,0.000011,0.001656
Malfunction,0.999990,1.000000,0.999844,0.000011,0.000000
Average,0.999969,0.999448,0.999600,0.000011,0.000552
Overall,0.999935,0.999549,0.999737,0.000011,0.000451


In [32]:
lstm = LSTMNetwork(input_size=10, hidden_size=512, output_size=4, num_layers=1, network_type='multiclass').to(device)
loss = lstm.fit(train_loader_sonata, test_loader_sonata, 20, optim.NAdam(lstm.parameters(), lr=0.0001), nn.CrossEntropyLoss())
predictions = lstm.predict(test_loader_sonata)

Epoch 1/20 - Train Loss: 0.1653, Test Loss: 0.0287
Epoch 2/20 - Train Loss: 0.0146, Test Loss: 0.0094
Epoch 3/20 - Train Loss: 0.0065, Test Loss: 0.0051
Epoch 4/20 - Train Loss: 0.0040, Test Loss: 0.0034
Epoch 5/20 - Train Loss: 0.0028, Test Loss: 0.0027
Epoch 6/20 - Train Loss: 0.0022, Test Loss: 0.0021
Epoch 7/20 - Train Loss: 0.0017, Test Loss: 0.0018
Epoch 8/20 - Train Loss: 0.0014, Test Loss: 0.0014
Epoch 9/20 - Train Loss: 0.0011, Test Loss: 0.0014
Epoch 10/20 - Train Loss: 0.0009, Test Loss: 0.0011
Epoch 11/20 - Train Loss: 0.0007, Test Loss: 0.0009
Epoch 12/20 - Train Loss: 0.0006, Test Loss: 0.0009
Epoch 13/20 - Train Loss: 0.0004, Test Loss: 0.0008
Epoch 14/20 - Train Loss: 0.0004, Test Loss: 0.0007
Epoch 15/20 - Train Loss: 0.0003, Test Loss: 0.0006
Epoch 16/20 - Train Loss: 0.0003, Test Loss: 0.0006
Epoch 17/20 - Train Loss: 0.0002, Test Loss: 0.0006
Epoch 18/20 - Train Loss: 0.0002, Test Loss: 0.0006
Epoch 19/20 - Train Loss: 0.0002, Test Loss: 0.0005
Epoch 20/20 - Train L

In [57]:
df_scores = calculate_scores_multiclass_classification(Y_test_sonata, predictions)

[[93705     0     1     0]
 [    0  6484     0     0]
 [    4     0  3620     0]
 [    0     0     0  3195]]
[93705  6484  3620  3195]
[4 0 1 0]


In [58]:
df_scores

,Accuracy,Recall,F1 Score,FPR,FNR
Benign,0.999953,0.999989,0.999973,0.000301,0.000011
Flooding,0.999953,1.000000,1.000000,0.000000,0.000000
Fuzzy,0.999953,0.998896,0.999310,0.000010,0.001104
Malfunction,0.999953,1.000000,1.000000,0.000000,0.000000
Average,0.999953,0.999721,0.999821,0.000078,0.000279
